# Examples of using the model inspection class and the profiling functions

The inspection class can gather model parameters, gradients, activations and activation gradients.

The profiling module provide information about cpu and GPU timing, memory usage and flops operations.

In [1]:
import torch

from torchtrainer.models import UNetCustom
from torchtrainer.util.inspector import Inspector
from torchtrainer.util.profiling import benchmark_model

# Dummy data. In a real application this should be a batch of the dataset.
batch = torch.rand(8, 1, 224, 224)
labels = torch.randint(0, 1000, (8,))

model = UNetCustom((3, 3, 3), (1, 1, 1), (16, 32, 64))
model.eval();

### Model inspection

In [2]:
# Check intermediate activations of the model, as well as gradients and parameters
insp = Inspector(model)
# For activations, we need to explicitly start tracking to set up the forward hooks.
insp.start_tracking_activations()

# Apply model to batch
res = model(batch)
# Dummy calculation of loss and gradients, just as an example.
loss = res.sum().backward()

# Remove the forward hooks
insp.stop_tracking_activations()

# Activations
acts = insp.get_activations()
# Parameters
params = insp.get_params()
# Gradients
grads = insp.get_grads()

names, values = zip(*acts)
print(names[:10])    # Names of the first 10 layers
print(values[0].shape)  # Size of the activations of the first layer

('unetcustom.stage_input.0', 'unetcustom.stage_input.1', 'unetcustom.encoder.stage_0.0.conv1', 'unetcustom.encoder.stage_0.0.bn1', 'unetcustom.encoder.stage_0.0.conv2', 'unetcustom.encoder.stage_0.0.bn2', 'unetcustom.encoder.stage_0.0.residual_adj.0', 'unetcustom.encoder.stage_0.0.residual_adj.1', 'unetcustom.encoder.stage_0.0', 'unetcustom.encoder.stage_0.1.conv1')
torch.Size([8, 16, 224, 224])


Tracking activations involves copying all the data from the GPU to the CPU (to preserve GPU memory), which is expensive. We can provide and aggregation function that will be applied to the data before copying

In [3]:
def agg_func(data, module_name, data_type):
    return torch.tensor([data.min(), data.max()])

insp = Inspector(model, agg_func=agg_func)
insp.start_tracking_activations()
res = model(batch)
insp.stop_tracking_activations()

acts = insp.get_activations()
print(acts)

[('unetcustom.stage_input.0', tensor([-0.7785,  0.8042])), ('unetcustom.stage_input.1', tensor([-0.7785,  0.8042])), ('unetcustom.encoder.stage_0.0.conv1', tensor([-0.5338,  0.6149])), ('unetcustom.encoder.stage_0.0.bn1', tensor([-0.5338,  0.6149])), ('unetcustom.encoder.stage_0.0.conv2', tensor([-0.9259,  0.6080])), ('unetcustom.encoder.stage_0.0.bn2', tensor([-0.9259,  0.6080])), ('unetcustom.encoder.stage_0.0.residual_adj.0', tensor([-0.6166,  0.6094])), ('unetcustom.encoder.stage_0.0.residual_adj.1', tensor([-0.6166,  0.6094])), ('unetcustom.encoder.stage_0.0', tensor([0.0000, 0.8452])), ('unetcustom.encoder.stage_0.1.conv1', tensor([-0.9459,  0.6439])), ('unetcustom.encoder.stage_0.1.bn1', tensor([-0.9459,  0.6439])), ('unetcustom.encoder.stage_0.1.conv2', tensor([-0.6720,  0.4935])), ('unetcustom.encoder.stage_0.1.bn2', tensor([-0.6720,  0.4935])), ('unetcustom.encoder.stage_0.1', tensor([0.0000, 1.0205])), ('unetcustom.encoder.stage_0.2.conv1', tensor([-0.8025,  0.9467])), ('une

We can also only track individual modules

In [4]:
insp = Inspector(model, [model.stage_input,model.encoder.stage_2[0]], agg_func)
insp.start_tracking_activations()
res = model(batch)
insp.stop_tracking_activations()

acts = insp.get_activations()
print(acts)

[('unetcustom.stage_input', tensor([0.0000, 0.8042])), ('unetcustom.encoder.stage_2.0', tensor([0.0000, 1.6012]))]


It is also possible to track activation gradients, but it is important to note that they do not work for a layer if the previous layer has an inplace operation.

In [5]:
insp = Inspector(model, [model.stage_input])
insp.start_tracking_act_grads()
res = model(batch)
insp.stop_tracking_act_grads()
loss = res.sum().backward()

act_grads = insp.get_act_grads()

### Model profiling

In [6]:
tensor_shape = (8, 1, 224, 224)
# Benchmark the model for training
stats_train = benchmark_model(
    model, tensor_shape, no_grad=False, call_backward=True, use_float16=True
    )
# Benchmark for inference
stats_val = benchmark_model(
    model, tensor_shape, no_grad=True, call_backward=False, use_float16=True
    )
# The units for each metric are also included in the dictionary
print(stats_train)
print(stats_val)

{'memory': 0.305875301361084, 'time_cpu': 0.004746437072753906, 'time_gpu': 0.027408063888549803, 'info': ['memory: GiB', 'time_cpu: s', 'time_gpu: s']}
{'memory': 0.1378645896911621, 'time_cpu': 0.0015673637390136719, 'time_gpu': 0.005228544235229492, 'info': ['memory: GiB', 'time_cpu: s', 'time_gpu: s']}
